In [2]:
import json

In [3]:
pile_uie_json_path = "../dataset/instruct_uie_ner_converted.json"

In [7]:
pile = json.load(open(pile_uie_json_path, "rb"))

In [72]:
def gen_entity_mentions_str(data):
    entities = data["entities"]
    entity_type_dict =dict()
    for item in entities:
        name = item["name"]
        _type = item["type"]
        name_list = entity_type_dict.get(_type,[])
        name_list.append(name)
        entity_type_dict[_type] = name_list 
    lines = []
    for k,v in entity_type_dict.items():
        lines.append("{}:[{}]".format(k, ",".join(list(set(v)))))
    return "\n".join(lines)

In [73]:
pile[1]

{'sentence': 'Juho Haapoja\n\nJuho Haapoja (born 22 October 1980) is a Finnish former professional boxer who competed from 2006 to 2017, and held the European Union cruiserweight title twice between 2011 and 2017. Professional career\nHaapoja made his professional debut on 8 April 2006, losing a four-round points decision to Remigijus Ziausys, who also debuted. A ten-round split draw against Jari Markkanen on 16 March 2007 denied Haapoja a chance to win the Finnish heavyweight title. He lost for a second time to Denis Bakhtov on 22 May 2008, which was for a WBC and PABA regional heavyweight title. On 28 November 2008, Haapoja stopped Sami Elovaara in nine rounds to become the Finnish heavyweight champion, winning the vacant title in his second attempt. A defence of this title against Jarno Rosberg, which went the ten-round distance, was later relegated to a no contest after Rosberg was caught for steroids. On 23 September 2011, Haapoja won his first major regional title by defeating Fa

In [74]:
entity_mention = gen_entity_mentions_str(pile[1])

In [75]:
print(entity_mention)

Person:[Faisal Ibnel Arrami,Sami Elovaara,Juho Haapoja,Jarno Rosberg,Jari Markkanen,Ian Tims,Mateusz Masternak,Remigijus Ziausys,Denis Bakhtov,Francesco Versaci]
Organization:[PABA,European Union,WBC]


In [10]:
system_prompt = """/no_think
You generate extraction-oriented label descriptions for multi-label NER.
Descriptions are used as prompts for span extraction models.
Do not include entity strings, examples, or quotes from the text.
Do not explain your reasoning.
Follow the output format strictly."""

In [124]:
gen_prompt = """Text:
{TEXT}

The following entity types appear in the text.
Each entity type may have multiple mentions.

For each entity type, write SIX short definitions that help extract its mentions.

Requirements:
- Definitions 1–5 MUST be grounded in THIS text.
- Definition 6 MUST be general and NOT depend on this text.
- All definitions for the same entity type must be semantically consistent.
- Focus on how the entity type is expressed in text, not on real-world ontology.
- Do NOT include any entity names or concrete examples.
- Keep each definition concise (20–45 tokens, Definition 6 can be 15–30 tokens).

Entity types and their mentions:
{entity_label_mentions}

Output format (strict JSON, one object per entity type):

""" 
post_fix_prompt = json.dumps([
  {
    "label": "<label_name>",
    "definitions": {
      "D1": "<canonical, extraction-oriented, context in this Text>",
      "D2": "<paraphrase of D1>",
      "D3": "<pattern-focused, how it appears in this text, true label should not appear.>",
      "D4": "<boundary-focused, include/exclude cues>",
      "D5": "<minimal, compressed version>",
      "D6": "<general definition, text-independent>"
    }
  }
], indent=4)

In [113]:
def generate_prompt(data):
    entity_mention_str  = gen_entity_mentions_str(data)
    # print(type(entity_mention_str))
    prompt = gen_prompt.format(TEXT = data["sentence"], entity_label_mentions=entity_mention_str)
    return prompt+post_fix_prompt

In [114]:
print(generate_prompt(pile[1]))

Text:
Juho Haapoja

Juho Haapoja (born 22 October 1980) is a Finnish former professional boxer who competed from 2006 to 2017, and held the European Union cruiserweight title twice between 2011 and 2017. Professional career
Haapoja made his professional debut on 8 April 2006, losing a four-round points decision to Remigijus Ziausys, who also debuted. A ten-round split draw against Jari Markkanen on 16 March 2007 denied Haapoja a chance to win the Finnish heavyweight title. He lost for a second time to Denis Bakhtov on 22 May 2008, which was for a WBC and PABA regional heavyweight title. On 28 November 2008, Haapoja stopped Sami Elovaara in nine rounds to become the Finnish heavyweight champion, winning the vacant title in his second attempt. A defence of this title against Jarno Rosberg, which went the ten-round distance, was later relegated to a no contest after Rosberg was caught for steroids. On 23 September 2011, Haapoja won his first major regional title by defeating Faisal Ibnel 

In [33]:
# gen_prompt = """Text:
# {TEXT}

# Entity label:
# {LABEL}

# Entity Example:
# {ENTITY}

# Write variety short label description grounded in the text that helps extract mentions of this label from similar texts.

# Constraints:
# - No examples and no entity names from the text.
# - 20–50 tokens.
# - Focus on how the label is expressed in text.

# Output format (exactly 5 lines):
# Definition1: <one sentence>
# Definition2: <one sentence>
# Definition3: <one sentence>
# Short Definition:<one sentence 5-10 tokens>
# Short General Definition:< one sentence ignore Text, common sense>"""


In [34]:
text = """Juho Haapoja\n\nJuho Haapoja (born 22 October 1980) is a Finnish former professional boxer who competed from 2006 to 2017, and held the European Union cruiserweight title twice between 2011 and 2017. Professional career\nHaapoja made his professional debut on 8 April 2006, losing a four-round points decision to Remigijus Ziausys, who also debuted. A ten-round split draw against Jari Markkanen on 16 March 2007 denied Haapoja a chance to win the Finnish heavyweight title. He lost for a second time to Denis Bakhtov on 22 May 2008, which was for a WBC and PABA regional heavyweight title. On 28 November 2008, Haapoja stopped Sami Elovaara in nine rounds to become the Finnish heavyweight champion, winning the vacant title in his second attempt. A defence of this title against Jarno Rosberg, which went the ten-round distance, was later relegated to a no contest after Rosberg was caught for steroids. On 23 September 2011, Haapoja won his first major regional title by defeating Faisal Ibnel Arrami via unanimous decision (UD) to become the European Union cruiserweight champion. He made two successful defences against Ian Tims on 21 January 2012 (twelve-round unanimous decision) and Francesco Versaci on 15 September 2012 (fourth-round stoppage), but lost the title to Mateusz Masternak on 15 December 2012 (unanimous decision)."""

In [35]:
label = """Person"""
entity = """Juho Haapoja"""

In [43]:
prompt = gen_prompt.format(TEXT=text, LABEL=label, ENTITY=entity)

In [45]:
prompt

'Text:\nJuho Haapoja\n\nJuho Haapoja (born 22 October 1980) is a Finnish former professional boxer who competed from 2006 to 2017, and held the European Union cruiserweight title twice between 2011 and 2017. Professional career\nHaapoja made his professional debut on 8 April 2006, losing a four-round points decision to Remigijus Ziausys, who also debuted. A ten-round split draw against Jari Markkanen on 16 March 2007 denied Haapoja a chance to win the Finnish heavyweight title. He lost for a second time to Denis Bakhtov on 22 May 2008, which was for a WBC and PABA regional heavyweight title. On 28 November 2008, Haapoja stopped Sami Elovaara in nine rounds to become the Finnish heavyweight champion, winning the vacant title in his second attempt. A defence of this title against Jarno Rosberg, which went the ten-round distance, was later relegated to a no contest after Rosberg was caught for steroids. On 23 September 2011, Haapoja won his first major regional title by defeating Faisal I

## 模型调用

In [39]:
import os
from openai import OpenAI

In [48]:
client = OpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx"
    api_key=os.getenv("DASHSCOPE_API_KEY","sk-b95c066723574c7ba72147797a568395"),
    base_url="https://dashscope.aliyuncs.com/api/v2/apps/protocols/compatible-mode/v1",
)

response = client.responses.create(
    model="qwen3-max",
    input=prompt,
    temperature = 0.8
)

# 获取模型回复
print(response.output_text)

Definition1: A Finnish former professional boxer active between 2006 and 2017, known for competing in cruiserweight and heavyweight divisions.  
Definition2: A retired boxer who twice held the European Union cruiserweight title and was also a national heavyweight champion.  
Definition3: A professional fighter whose career included regional title bouts and notable matches ending in draws, losses, and stoppages.  
Short Definition: Former Finnish professional boxer and regional titleholder.  
Short General Definition: A person who competed professionally in boxing.


In [125]:
prompt = generate_prompt(pile[1])

In [126]:
print(prompt)

Text:
Juho Haapoja

Juho Haapoja (born 22 October 1980) is a Finnish former professional boxer who competed from 2006 to 2017, and held the European Union cruiserweight title twice between 2011 and 2017. Professional career
Haapoja made his professional debut on 8 April 2006, losing a four-round points decision to Remigijus Ziausys, who also debuted. A ten-round split draw against Jari Markkanen on 16 March 2007 denied Haapoja a chance to win the Finnish heavyweight title. He lost for a second time to Denis Bakhtov on 22 May 2008, which was for a WBC and PABA regional heavyweight title. On 28 November 2008, Haapoja stopped Sami Elovaara in nine rounds to become the Finnish heavyweight champion, winning the vacant title in his second attempt. A defence of this title against Jarno Rosberg, which went the ten-round distance, was later relegated to a no contest after Rosberg was caught for steroids. On 23 September 2011, Haapoja won his first major regional title by defeating Faisal Ibnel 

In [127]:
import os
import dashscope

messages = [
    {'role': 'system', 'content': system_prompt},
    {'role': 'user', 'content': prompt}
]
response = dashscope.Generation.call(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx"
    api_key=os.getenv('DASHSCOPE_API_KEY',"sk-b95c066723574c7ba72147797a568395"),
    model="qwen-plus", # 此处以qwen-plus为例，可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    messages=messages,
    result_format='message'
    )


In [128]:
print(response)

{"status_code": 200, "request_id": "28c7f2fa-a909-46e7-b034-e38781c92e36", "code": "", "message": "", "output": {"text": null, "finish_reason": null, "choices": [{"finish_reason": "stop", "message": {"role": "assistant", "content": "[\n    {\n        \"label\": \"Person\",\n        \"definitions\": {\n            \"D1\": \"A proper noun phrase appearing as the subject or object of verbs denoting boxing-related actions, such as 'debuted', 'lost', 'defeated', 'stopped', or 'won', and often followed by dates, titles, or event descriptions.\",\n            \"D2\": \"A capitalized name that functions syntactically as an argument of boxing events—e.g., fights, title contests, defenses—or career milestones like debuts and retirements.\",\n            \"D3\": \"A sequence of one or two capitalized words, typically adjacent to boxing-specific predicates or modifiers (e.g., 'vs.', 'against', 'to', 'defeated', 'lost to'), without intervening punctuation or descriptors.\",\n            \"D4\": \"I

In [129]:
print(response["output"]["choices"][0]["message"]["content"])

[
    {
        "label": "Person",
        "definitions": {
            "D1": "A proper noun phrase appearing as the subject or object of verbs denoting boxing-related actions, such as 'debuted', 'lost', 'defeated', 'stopped', or 'won', and often followed by dates, titles, or event descriptions.",
            "D2": "A capitalized name that functions syntactically as an argument of boxing events—e.g., fights, title contests, defenses—or career milestones like debuts and retirements.",
            "D3": "A sequence of one or two capitalized words, typically adjacent to boxing-specific predicates or modifiers (e.g., 'vs.', 'against', 'to', 'defeated', 'lost to'), without intervening punctuation or descriptors.",
            "D4": "Include only the bare name; exclude titles (e.g., 'Dr.', 'Mr.'), honorifics, nicknames, or appended roles—boundaries end before commas, parentheses, or prepositions introducing non-name material.",
            "D5": "Capitalized name phrase directly involved in 

In [62]:
print(response["output"]["choices"][0]["message"]["content"])

Definition1: Refers to individuals identified by full names, often with biographical details such as birth dates, nationalities, professions, or career timelines.  
Definition2: Includes people mentioned in contexts of professional achievements, titles held, competitive records, or chronological career milestones.  
Definition3: Typically appears as a named subject of sentences describing actions, roles, accomplishments, or relationships within sports, occupations, or historical events.  
Short Definition: Named individual with personal and professional attributes.  
Short General Definition: A human being identified by name in narrative or factual text.


In [58]:
prompt

'Text:\nJuho Haapoja\n\nJuho Haapoja (born 22 October 1980) is a Finnish former professional boxer who competed from 2006 to 2017, and held the European Union cruiserweight title twice between 2011 and 2017. Professional career\nHaapoja made his professional debut on 8 April 2006, losing a four-round points decision to Remigijus Ziausys, who also debuted. A ten-round split draw against Jari Markkanen on 16 March 2007 denied Haapoja a chance to win the Finnish heavyweight title. He lost for a second time to Denis Bakhtov on 22 May 2008, which was for a WBC and PABA regional heavyweight title. On 28 November 2008, Haapoja stopped Sami Elovaara in nine rounds to become the Finnish heavyweight champion, winning the vacant title in his second attempt. A defence of this title against Jarno Rosberg, which went the ten-round distance, was later relegated to a no contest after Rosberg was caught for steroids. On 23 September 2011, Haapoja won his first major regional title by defeating Faisal I

## 统计 pile 中的label 分布